# Evaluation History - Segmentation Masks

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

import os
import numpy as np

from src.heatmaps.evaluation import HeatmapEvaluationHistory

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['image.cmap'] = 'viridis'

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 15})

In [9]:
notebook_key = 'captum--guided-grad-cam-insertion-TP-TN-FP-FN'
# notebook_key = 'captum--guided-grad-cam-deletion-TP-TN-FP-FN'
# notebook_key = 'captum--guided-grad-cam--eval-100-insertion-TP-TN-FP-FN'

history = HeatmapEvaluationHistory.load('../../tmp/risei-history', notebook_key)